### Simple LLM with Langchain Expression Language

In [1]:
# Load env data
import os
from dotenv import load_dotenv
load_dotenv()

# Access GROQ API KEY
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x797fafc09d60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x797fafeb2840>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello, How are you?")
]

result = model.invoke(messages)
result

AIMessage(content='Bonjour, Comment allez-vous ? \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 22, 'total_tokens': 34, 'completion_time': 0.021818182, 'prompt_time': 0.00014859, 'queue_time': 0.020536148, 'total_time': 0.021966772}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-712f1f24-1433-4a95-958b-84b3963a7165-0', usage_metadata={'input_tokens': 22, 'output_tokens': 12, 'total_tokens': 34})

In [9]:
# Apply Output parser
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(result)

'Bonjour, Comment allez-vous ? \n\n'

In [11]:
# Create chain
chain = model | output_parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\nThis is a formal way to greet someone. \nHere are some other options:\n\n* **Informal:** Salut, ça va ?  \n* **Very informal:** Hey, comment tu vas ? \n\n'

### Prompt Template

In [13]:
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}:'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [15]:
result = prompt.invoke({"language": "Hindi", "text": "Hello" })
result.to_messages

<bound method ChatPromptValue.to_messages of ChatPromptValue(messages=[SystemMessage(content='Translate the following into Hindi:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])>

In [ ]:
# Chain component with LCEL
chain = prompt | model | output_parser
chain.invoke({"language": "Hindi", "text": "Hello, How are you?" })

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nThis is a common and polite way to greet someone in Hindi. \n'